# Tarea de programación:
# Support Vector Machines (Máquinas de Soporte Vectorial)

## Introducción

En esta sección usted descubrirá aplicaciones de las máquinas de soporte vectorial (SVM) para detectarm "spam email". Es decir, correos no deseados. Antes de comenzar recomendamos revisar los videos y notas de clase en el tópico.

Toda la información que usted necesita para resolver esta tarea está incluida en este cuaderno, incluyendo los programas en `Python`.

Antes de comenzar con el ejercicio es bueno que tenga instaladas las librerias que se necesitan para el mismo.
Usaremos [`NumPy`](http://www.numpy.org/)  para todos las operaciones entre arreglos y matrices, [`matplotlib`](https://matplotlib.org/) para las gràficas, y  [`scipy`](https://docs.scipy.org/doc/scipy/reference/) para cálculos
numéricos científcos. 


In [ ]:
# se usa para el manejo de caminons
import os

# para calculos numericos
import numpy as np

# expresiones regulares para manipular emails
import re

# para graficación
from matplotlib import pyplot

# para optiización
from scipy import optimize

# para cargar datos en formato MATLAB
from scipy.io import loadmat

# funciones adicionales que se usan en esta tarea
import utils


# las graficas se hacen internamente en este cuaderno
%matplotlib inline

## 1 Support Vector Machines (Máquinas de Soporte Vectorial)
En esta primera mitad del ejecicio ysted va a usar SVMs con varios datos 2D. La experimentaciòn
con estos datos le ayudará a obtner intuición en cnmo las SVMs trabajan y el uso de un kerne Gaussiano con SVMs.
En la segunda mitad del ejercicio usted usará SVMs para desarrollar una aplicación que detecte mensajes no deseados
(spam emails). 

### 1.1 Datos para el ejemplo 1

Comenzamos con un ejemplo en 2D el cual puede ser linelamente separado por una frontera lineal. La siguiente celda
grafica los datos de entrenamiento que se deben ver como


![Dataset 1 training data](Figures/dataset1.png)

En estos datos las posiciones de las muestras positivas indicadas con `x` ) y las negativas (indicadas con `o`) sugieren
una separación nantural indicadad por el margen interno. Sin embargo, note que hay un punto anómalo (indicado con `x`) a la izquierda
ceca a (0.1, 4.1). Como parte de este ejercicio usted entenderá como esta muestra anómala afecta la frontera de decisión del SVM.

In [ ]:
# carge los datos
# obtiene X, y . Los datos vienen en forma de diccionario con llaves X,y
data = loadmat(os.path.join('Data', 'ex6data1.mat'))
X, y = data['X'], data['y'][:, 0]

# grafique los datos
utils.plotData(X, y)

En esta parte del ejercicio usted debe usar distintos valores de $C$. Informalmente el parámetro $C$ es un valor positivo que controla la clasificación, penalizando muestras anómalas. Un valor grande de $C$ le dice al SVM que trate de clasificar todas las muestras
correctamente. $C$ se comporta como $1/\lambda$ en regularización como la hemos venido discutiendo en este curso.

La siguiente celda corre el entrenamiento de SVM (con $C=1$) con software que está incluido con esta tarea (la función  `svmTrain` 
en el mòdulo `utils` ). Cuando$C=1$, usted debería encontrar la frontera de decisión en la margen entre los dos grupos y
una *clasificacion incorrecta* del punto en el extremo izquierdo, tal como se muestra en la figura (izquierda) abajo. 

<table style="text-align:center">
    <tr>
        <th colspan="2" style="text-align:center">SVM Decision boundary for example dataset 1 </th>
    </tr>
    <tr>
        <td style="text-align:center">C=1<img src="Figures/svm_c1.png"/></td>
        <td style="text-align:center">C=100<img src="Figures/svm_c100.png"/></td>
    </tr>
</table>

<div class="alert alert-block alert-warning">
Con el objeto de minimizar la dependencia de esta tarea, de las librerías externas, incluimos con esta iplementación, el algoritmo `utils.svmTrain` .  Sin embargo, esta implementación en particular no es muy eficiente,. Si usted está entrenando SVM en un problema real, especialmente si necesita uno conjunto de datos grande, le recomendamos usar herramientas de SVM altamente optimizadas, tales como[LIBSVM](https://www.csie.ntu.edu.tw/~cjlin/libsvm/). La librería de machine leanring de Python [scikit-learn](http://scikit-learn.org/stable/index.html) provides wrappers for the LIBSVM library.
</div>
<br/>
<div class="alerta">
**Nota de implementación**: La mayoría de paquetes (incluido la función `utils.svmTrain`) automáticamente agrega el atributo (corresondiente al intercepto $\theta_0$)  $x_0$ = 1 automàticamente. Particularmente, en Python usted debe trabajar con muestra de entrenamiento $x \in \mathcal{R}^n$ (en vez de  $x \in \mathcal{R}^{n+1}$); Por ejemplo, en el primer ejemplo os datos se encuentran en   $x \in \mathcal{R}^2$.
</div>

Su tarea es tratar diferentes valores de $C$ con estos datos. Específicamente, usted debe cambiar los valores de $C$ en la próxima celda a $C=100$ y correr el entrenamiento de SVM de nuevo. Cuando $C=100$ usted debería observar que el SVM clasifica todas las muestras correctamente y aparece como un ajuste natural a los datos 

In [ ]:
# cambie C y observe los resultados
# la frotera varía  (por ejemplo, trate con C = 1000)
C = 1

model = utils.svmTrain(X, y, C, utils.linearKernel, 1e-3, 20)
utils.visualizeBoundaryLinear(X, y, model)

<a id="section1"></a>
### 1.2 SVM  con kernels Gaussianos.
En esta parte del ejercicio usted va a usar SVMs para realizar una clasificación no lineal. En particular, usted va a usar un kernel Gaussiano en datos que no son 
separables linealmente. 


#### 1.2.1 Kernel Gaussiano

Para encontrar una frontera de decición no lineal en SVM se necesita implementar primero un kernel Gaussiano. Puede pensar en un kernel Gaussiano como una funcion de similaridad de
medidas de "distancia" entre un par de muestras, ($x^{(i)}$, $x^{(j)}$). El kernel Gaussiano tambien se parametriza con el parámetro de ancho de banda $\sigma$, el cual determina
que tan rápido la medida de similaridad se decrece (a 0) a medida que las muestras se separan. 
Usted debe completar el código  `gaussianKernel` para calcular el kernel Gaussiano entre dos muetras ($x^{(i)}$, $x^{(j)}$).  El kernel Gaussiano se define como

$$ K_{\text{gaussian}} \left( x^{(i)}, x^{(j)} \right) = \exp \left( - \frac{\left\lvert\left\lvert x^{(i)} - x^{(j)}\right\lvert\right\lvert^2}{2\sigma^2} \right) = \exp \left( -\frac{\sum_{k=1}^n \left( x_k^{(i)} - x_k^{(j)}\right)^2}{2\sigma^2} \right)$$
<a id="gaussianKernel"></a>

In [ ]:
def gaussianKernel(x1, x2, sigma):
    """
    Calcula la funcion de base radial
    Retornal la funcion de base radial entre x1 y x2.
   
    
    Parámetros
    ----------
    x1 :  arreglo en numpy
        Un vector de tamaño (n, ), como punto inicial. 
    
    x2 : arreglo en numpy
        Un arreglo de tamaño (n, ) como punto final. 
    
    sigma : float
        Parámetro de ancho de banda para el kernel Gaussiano. 

    Retorna
    -------
    sim : float
        El computo de la funcion de base radial RBF (radial base function)
        entre los dos puntos datos. 
    
    Instrucciones
    ------------
    Llene esta función y retorne la similaridad entre los puntos  `x1` y `x2`
    mediante el cómputo del kernel Gaussiano con ancho de banda `sigma`.
    """
    sim = 0
    # ====================== SU CÓDIGO ACÁ ======================



    # =============================================================
    return sim

Una vez completada la función `gaussianKernel` la próxima celda prueba esta función con dos muestras datas.
Debería obtener un valor de 0.324652.

In [ ]:
x1 = np.array([1, 2, 1])
x2 = np.array([0, 4, -1])
sigma = 2

sim = gaussianKernel(x1, x2, sigma)

print('Gaussian Kernel between x1 = [1, 2, 1], x2 = [0, 4, -1], sigma = %0.2f:'
      '\n\t%f\n(for sigma = 2, this value should be about 0.324652)\n' % (sigma, sim))

### 1.2.2 Ejemplo 2

En la próxima parte de este cuaderno usted debe cargar el segundo grupo de datos, 
que se muestra en la siguiente figura. 

![Dataset 2](Figures/dataset2.png)

In [ ]:
# Gargue ex6data2
# Obtenga X, y como llaves en los datos en forma de diccionario.
data = loadmat(os.path.join('Data', 'ex6data2.mat'))
X, y = data['X'], data['y'][:, 0]

# Grafica 
utils.plotData(X, y)

De la figura se puede ver que la frontera de decisión, que separa los puntos positivos de los negativos,  no es lineal. Sin embargo, mediante el uso del kernel Gaussiano con SVM, usted
será capaz de aprender la frontera de decisión no lineal la cual se comporta bien para los datos. Si usted implementa correctamente el kernel Gaussiano, la siguiente celda
procederá a entrenar el SVM con el kernel Gaussiano en los datos. 

Usted debería obtener una frontera de decisión como se muestra abajo, tal y como se calcula con un kernel Gaussiano. La frontera de decisión debe ser capaz de separar la mayoría de las
muestras positivas de las negativbas correctamente y seguir los contornos de los datos adecuadamente. 


![Dataset 2 decision boundary](Figures/svm_dataset2.png)

In [ ]:
# parámetros de SVM
C = 1
sigma = 0.1

model= utils.svmTrain(X, y, C, gaussianKernel, args=(sigma,))
utils.visualizeBoundary(X, y, model)

<a id="section2"></a>
#### 1.2.3 Ejemplo 3

En esta parde del ejercicio usted desarrollará habilidades prácticas acerca del uso del kernel Gaussiano en SVM. La próxima celda carga y muestra el tercer grupo
de datos, el cual debe lucir como se muestra en la figura siguiente. 



![Dataset 3](Figures/dataset3.png)

Usted usará este conjunto de datos con un kernel Gaussiano en SVM. En los datos suministrados  , `ex6data3.mat`, usted obtiene las variables: 
 `X`, `y`, `Xval`, `yval`. 

In [ ]:
# cargue los datos ex6data3
# obtiene las variables X, y, Xval, yval como llaves de un diccionario
data = loadmat(os.path.join('Data', 'ex6data3.mat'))
X, y, Xval, yval = data['X'], data['y'][:, 0], data['Xval'], data['yval'][:, 0]

# grafique los datos
utils.plotData(X, y)

Su tarea es usar los conjuntos de validación cruzada `Xval`, `yval` para determinar el mejor $C$ y $\sigma$.  Debe escribir cualquier código adicional
que le ayude a encontrar los parámetros  $C$ y $\sigma$. Para ambos  $C$ y $\sigma$, sugerimos tratar valores de forma multiplicativa (por ejemplo:  0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30).
Es importante tratar todos los posibles pares de $C$ y $\sigma$ (for ejemplo $C=0.3$ y $\sigma=0.1$.  Si trata los 8 valores posibles de $C$ y $\sigma$, obtiene un total entrenamiento
y evalucación (sobre el conjunto de validación) de $8^2 = 64$ modelos diferentes. Luego de determinar los $C$ y $\sigma$ óptimos, debe modificar el código  `dataset3Params`, completando
con los parámetros encontrados. Para nuestros mejores parámetros la frontera de decisión se muestra en la figura siguiente. 


![](Figures/svm_dataset3_best.png)

<div class="alert alert-block alert-warning">
**Pista de implementación:** Cuando implementa la validación cruzada para seleccionar los parámetros óptimos de $C$ y $\sigma$, necesita evaluar el error en los datos de validación cruzada.
Recuerde que para la clasificación, el error se define como la fracción de muestras, de validación cruzada, clasificadas incorrectmente. En `numpy`, usted puede calcular el error usando
`np.mean(predictions != yval)`, donde `predictions` es el vector que contiene las predicciones del SVM, y  `yval` son las etiquetas tomadas del conjunto de validación  cruzada. Usted puede
usar la herramienta `utils.svmPredict` para generar predicciones en el caso del conjunto de validación cruzada. 
    
</div>
<a id="dataset3Params"></a>

In [ ]:
def dataset3Params(X, y, Xval, yval):
    """
    Retorna su escogencia de C y sigma para la parte 3 de este ejercicio
    donde usted debe seleccionar los C y sigma óptimos para el uso de
    SVM con un kernel RBF. 
    
    
    Parámetros 
    ----------
    X : arreglo
        matriz (m x n) de datos de entrenamiento donde m es el número de
        muestras de entrenamiento, y n es el número de atributos. 
        
    
    
    y : arreglo
        vector (m, ) vector de eiquetas para los datos de entrenamiento
    
    Xval : arreglo
        matriz (mv x n) de datos de validacion, donde mv es el número de 
        muestras de validación y n es el número de atributos. 
       
    
    yval : arreglo
        vector (mv, ) de etiquetas de validación
    
    Returna
    -------
    C, sigma : float, float
        Los valores óptimos de parámetro de regularización C y
        parámetro sigma de la RBF. 
       
    
    Instructiones
    ------------
    Llene esta función que retorna los parámetros  C y sigma
    mediante el uso de validación cruzada.  
    Usted puede usar  `svmPredict` con el fin de predecir las
    etiquetas en el conjunto de validación cruzada. Por ejemplo, 
    
  
    
        predicciones = utils.svmPredict(model, Xval)

    retornara las predicciones para el conjunto de validación cruzada. 
    
    Nota
    ----
    Usted puede calcular el error de predicción mediante el uso de
    
        np.mean(predictions != yval)
    """
    # Necesita retornar las siguientes variables correctamente. 
    C = 1
    sigma = 0.3

    # ====================== SU CÓDIGO ACÁ ======================

    
    
    # ============================================================
    return C, sigma

El código en la próxima celda entrena un clasificador SVM usando el conjunto de entrenamiento $(X, y)$ y los parámetros cardados desde `dataset3Params`. Note que esto puede tomar
algunos minutos en ejecutarse. 

In [ ]:
# Trate differentes parámetros SVM acá
C, sigma = dataset3Params(X, y, Xval, yval)

# entrene el SVM
# model = utils.svmTrain(X, y, C, lambda x1, x2: gaussianKernel(x1, x2, sigma))
model = utils.svmTrain(X, y, C, gaussianKernel, args=(sigma,))
utils.visualizeBoundary(X, y, model)
print(C, sigma)

<a id="section3"></a>
## 2 Clasificación de mensajes no deseados (spam email)



Muchos servicios de mensajes hoy proveen filtros contra mensajes no deseados, los cuales clasifican los mensajes como "buenos" (aceptados) y "malos" (rechazados).
En esta parte del ejercicio usted usará SVM para constuir su propio filtro contra mensajes no deseados. 

Usted debe entrenar el calsificador para decidir si el mensaje $x$ es "spam" ($y=1$) o "no-spam" ($y=0$). En particular, usted necesita convertir cada mensaje en
un atributo $x \in \mathbb{R}^n$. Las siguientes partes del ejercicio le indicarán como se debe construir el vector atributo para cada "email".

Los datos incluidos en este ejercicio se basan en un subconjunto de  [SpamAssassin Public Corpus](http://spamassassin.apache.org/old/publiccorpus/). 

Para este ejercicio usted solo usará el cuerpo del mensaje (se excluyen los encabezados).

### 2.1 Procesamiento de mensajes
Antes de comenzar esta tarea recomendamos que mire los ejemplos de datos. La figura siguiente muestra una musestra de mensaje que contiene una URL, una direccion de "email" (al final), números y una cantidad en dólares. 

<img src="Figures/email.png" width="700px" />

Mientras muchos mensajes pueden contener eventos similares (por ejemplo, números, otros URLs o otras direcciones de mensajes), las entidades específicas (por ejemplo, un URL específico o una
cantidad en dólares) debe ser diferente en casi culquier otro "email". Por esto, un método usualmente empleado en procesamiento de mensajes es la "normalización" de estos valores de forma
que todas las URLs son tratados igualmente, todos los números son tratados igualmente, etc.  Por ejemplo, podríamos reemplazar cada URL en el mensaje con un cadena única "httpaddr" para indicar
que una URL está presente. 

Esto tiene el efecto de permitir que el clasificador tome una decisión con base a si una URL está presente sin importar cual URL. Esto típicamente mejora la desempeño de un clasificador, dado
que los mensajes indeseados usualmente aleatorizan los URLs, de forma que encontrar un URL particular de nuevo en un mensaje se hace escazo.

En la función  `processEmail` abajo, implementamos los siguientes pasos de preproceso y normalización. 


- **minúscula**: El mensaje en su totalidad se convierte a minúsculas, de forma que las mayúsculas son ignoradas (por ejemplo, IndICar se trata lo mismo que Indicar).


- **remoción de  HTML**: Todas las etiquetas HTML son removidas de los mensajes. Muchos mensajes a menudo vienen con un formato HTML ; removemos todas las etiquetas HTML, de forma
que solo quede el contenido. 


- **Normalización de  URLs**: Todas las URLs se reemplazan con el texto "httpaddr". 


- **Normalización de direcciones de correo**:  todas las direcciones de correo se reemplazan con  “emailaddr”.

- **Normalización de  Números**: Todos los números se reemplazan con el texto  “number”.

- **Normalización de  Dólares**: Todos los signos de dolar ($) se reemplazan con el texto “dollar”.

- **Reducción a la raíz de las palabras**: Las palabras se reducen a su raíz. Por ejemplo, "descuento", "descontado", "descontando" todas se reemplazan con "descuento". Algunas
veces el depurador remueve caracteres adicionales del final, por ejemplo 
 “include”, “includes”, “included”, e “including”, todos ellos, se reemplazan por  “includ”.

- **eliminación de objetos que no son palabras**: Objetos que no son palabras y puntación se remueven. Todos los espacioes en blanco se reducen a un solo caracter en blanco.

El resultado de este preprocesamiento se muestra en la siguiente figura. 

<img src="Figures/email_cleaned.png" alt="email cleaned" style="width: 600px;"/>

Dado que el preprocesamiento deja fragmentos de palabras y objetos que no son palabras, esta forma se hace más conveniente para trabajar con la extración de atributos.

#### 2.1.1 Lista de vocabulario

Luego de procesar los correos tenemos una lista de palabras para cada uno. El próximo paso es escoger cuales palabras quisieramos tener en nuestro clasificador y cuales no. 

Para este ejercicio, solo escogemos las palabras más frecuentes (lista de vocabulario). Dado que las palabras escazas solo ocurren en unos pocos mensajes, estas tienen a sobreajustar
el modelo. La lista completa de vocabulario está en el archivo  `vocab.txt` (dentro del directorio `Data` para este ejercicio). Estas se muestran en la siguiente figura. 



<img src="Figures/vocab.png" alt="Vocab" width="150px" />

Nuestro vocabulario se seleccionó con palabras que ocurren por lo menos 100 veces en el cuerpo de correos no deseados (spam corpus), 
resultando en una lista de 1899 palabras. En la práctica, un vocabulario tiene mas o menos entre 10,000 y 50,000 palabras que son usadas
a menudo. Dado un vocabulario, podemos hacer un mapeo de cada palabra en los mensajes preprocesados hacia una lista de índices de palabras que contienen el índice de la
palabra en el diccionario. La siguiente figura muestra el mapeo para una muestra de mensaje. Específicamente, en la muestra, la palabra "anyone" se normaliza inicialmente
como "anyon" y luego se mapea al índice 86 de la lista. 

<img src="Figures/word_indices.png" alt="word indices" width="200px" />

Su tarea ahora es completar el códicog en la función  `processEmail` para crear este mapeo. En el código usted recibe una cadena   `word` con una sola palabra del
mensaje procesado. Debería buscar la palabra en la lista   `vocabList`. Si la palabra existe en la lista usted debe agregar un índice de la palabra en la variable 
 `word_indices` variable. Si la palabra no existe, y no está en el vocabulario, simplemente la salta. 
 
 
<div class="alert alert-block alert-warning">
**ayuda en python**: En python usted puede encontrar el índice de la primera ocurrencia de un objeto en  `list` mediante el uso del atributo  `index`. En el código suministado para  `processEmail`, `vocabList` hay una lista en python que contiene el vocabulario. Para entontrar el índice de la palabra podemos usar  `vocabList.index(word)` que retorna el número indicando el índice
de la palabra en la lista. Si la palabra no existe en la lista, un error de excepción `ValueError`  resulta. Entonces podemos usar la instrucción  `try/except` para capturar las excepciones  con el objeto de no parar el proceso. Piense en las instrucciones  `try/except` como del tipo  `if/else` , solo que ahora pregunta por perdón y no por permiso. 

Por ejemplo:
<br>

```
try:
    haga cosas aca
except ValueError:
    pass
    # no haga nada (perdóname) si un ValueError ocurre bajo el rótulo "try".
```
</div>
<a id="processEmail"></a>

In [ ]:
def processEmail(email_contents, verbose=True):
    """
    Preprocesa el cuerpo del mensaje y retorna una lista de índices
    de las palabras del mensaje. 
   
    
    Parámetros
    ----------
    email_contents : str
        Una cadena con el email. 
    
    verbose : bool
        Si True, imprime los resultados luego del proceso. 
    
    Retorna
    -------
    word_indices : list
        Lista de enteros que representan los índices de cada palabra
        en el mensaje, la cual también existe en el vocabulario. 
       
    
    Instrucciones
    ------------
    Complete esta función con el objeto de agregar el índice de 
    la palara a word_indices, siempre y cuando esté en el vocabulario.  

    En este punto, usted tiene una palabra raiz del mensaje en la variable word. 
    Usted debería buscar la palabra en el vocabuario (vocabList). 
    Si existe, debe agregar el índice de la palabra a la lista  word_indices.
    Concretamente, si la palabra = 'action', debería buscar esta en el
    vocabulario. For ejemplo, si  vocabList[18] =
    'action', debe agregar 18 al vector  word_indices 
    (por ejemplo, word_indices.append(18)).
    
    Notas
    -----
    - vocabList[idx] retorna la palabra con indice idx en el vocabulario
    
    - vocabList.index(word) retorna el índice de   `word` en el vocabulario.
     (una excepción de ValueError aparece si la palabra no existe.)
    """
    # cargue el vocabularoi
    vocabList = utils.getVocabList()

    # Inicialice el retorno
    word_indices = []

    # ========================== Preproceso del mensaje  ===========================
    # Encuentre encabezados y remuévalos  ( \n\n y remueva )
    # remueva el comentario de las  líneas siguientes si está trabajando con mensajes 
    # con encabezados
    # hdrstart = email_contents.find(chr(10) + chr(10))
    # email_contents = email_contents[hdrstart:]

    # convierta a minúsculas
    email_contents = email_contents.lower()
    
    # remueva todo el HTML
    # Busca una expresión que comienza con <  y termina con  > y reemplace
    # y que no tenga < o > en la etiqueta con espacio
    email_contents =re.compile('<[^<>]+>').sub(' ', email_contents)

    # Manipule números
    # Busque caracteres entre 0 y 9 
    email_contents = re.compile('[0-9]+').sub(' number ', email_contents)

    # Manipule URLS
    # busque cadenas que comienzan con
    # http:// or https://
    email_contents = re.compile('(http|https)://[^\s]*').sub(' httpaddr ', email_contents)

    # Manipule direcciones de email 
    # busque @ en el medio
    email_contents = re.compile('[^\s]+@[^\s]+').sub(' emailaddr ', email_contents)
    
    # manipule el signo $
    email_contents = re.compile('[$]+').sub(' dollar ', email_contents)
    
    # remueva puntuación
    email_contents = re.split('[ @$/#.-:&*+=\[\]?!(){},''">_<;%\n\r]', email_contents)

    # remueva cualquier cadena vacía 
    email_contents = [word for word in email_contents if len(word) > 0]
    
    # procese para la raiz de cada palabra
    stemmer = utils.PorterStemmer()
    processed_email = []
    for word in email_contents:
        # Remueva los caracteres no alfanuméricos 
        word = re.compile('[^a-zA-Z0-9]').sub('', word).strip()
        word = stemmer.stem(word)
        processed_email.append(word)

        if len(word) < 1:
            continue

        # Busque, en el diccionario y agregue a word_indices, si la encuentra
        # ====================== SU CÓDIGO ACÁ ======================

        

        # =============================================================

    if verbose:
        print('----------------')
        print('Processed email:')
        print('----------------')
        print(' '.join(processed_email))
    return word_indices

Una vez usted halla implementado `processEmail`, corra la siguiente celda y debería ver la salida del mensaje procesado y los la lista de índices mapeados. 

In [ ]:
# Use SVM para clasificar mensaes en Spam versus Non-Spam. Primero usted debe
# convertir cada mensaje en un vector de atributos. En esta parte, usted
# implementará los pasos de preproceso para cada mensaje. Debe completar
# el código en  processEmail.m para generar el vector de índices para
# un mensaje dado. 

# Extraiga Atributos
with open(os.path.join('Data', 'emailSample1.txt')) as fid:
    file_contents = fid.read()

word_indices  = processEmail(file_contents)

#Print Stats
print('-------------')
print('Word Indices:')
print('-------------')
print(word_indices)

<a id="section4"></a>
### 2.2 Extracción de atributos de los mensajes
En esta sección debe implementar la extracción que convierte cada mensaje en un vector en $\mathbb{R}^n$. Para este ejercicio, used debe usar n = # palabras en el vocabulario.
Específicamente, el atributo $x_i \in \{0, 1\}$ para un mensaje corresponde al hecho de que la palabra $i^{th}$ ocurren en el mensaje. Es decir, $x_i =1$ si la  palabra
$i^{th}$ está en el mensaje y $x_i =0$ si la palabra $i^{th}$ no está en el mensaje. 

Es decir, para un mensaje típico, este atributo debe verse como
$$ x = \begin{bmatrix} 
0 & \dots & 1 & 0 & \dots & 1 & 0 & \dots & 0 
\end{bmatrix}^T \in \mathbb{R}^n
$$

Debe, ahora, completar el código en la función  `emailFeatures` para generar un vector para el mensaje, dado el vector
 `word_indices`.
<a id="emailFeatures"></a>

In [ ]:
def emailFeatures(word_indices):
    """
    Produce un vector de atributos dado un vector word_indices 
    
    Parámetros
    ----------
    word_indices : list
        Lista con los índices de las palabras del vocabulario 
    
    Retorna
    -------
    x : list 
        El vector calculado. 
    
    Instrucciones
    ------------
    Complete esta función para que retorne un vector para un
    mensaje dado (índices de palabras). Para facilitar el proceso
    ya pre-procesamos el mensaje y convertimos cada palabra en un índice
    en un diccionario (de 1899 palabras). La variable 
     `word_indices` contiene una lista de índices de palabras que ocurren
     en un mensaje. 
     
     
    Concretamente, si un mensaje  tiene el texto: 
    
        The quick brown fox jumped over the lazy dog.
        
    El vector de índices para este texto debe verse como:

               
        60  100   33   44   10     53  60  58   5

    donde, cada palabra fue mapeada, por ejemplo:

        the   -- 60
        quick -- 100
        ...

    Nota
    ----
    Los números de arriba son solo un ejemplo. No son el mapeo real. 

    Su trabajo consiste en tomar cada vector de   `word_indices` y 
    construir un vector binario que indique si una palabra particular
    ocurre en el mensaje. Es decir,  x[i] = 1 cuando la palabra i
    está en el mensaje. Concretamente, si la palabra  'the' (con 
    índice 60) aparece en el email, entonces  x[60] = 1. El
    vector de atributo debe verse como
    
        x = [ 0 0 0 0 1 0 0 0 ... 0 0 0 0 1 ... 0 0 0 1 0 ..]
    """
    # Numero total de palabras en el vocabulario
    n = 1899

    # Necesita retornar esta variable correctamente. 
    x = np.zeros(n)

    # ===================== SU CÓDIGO ACÁ ======================

    
    
    # ===========================================================
    
    return x

Una vez implementado el código  `emailFeatures`,  corra la próxima celda. Debería encontrar que el vector atributo tiene una longitud de 1899 y 45 componentes no nulas. 

In [ ]:
# Extraiga Atributos
with open(os.path.join('Data', 'emailSample1.txt')) as fid:
    file_contents = fid.read()

word_indices  = processEmail(file_contents)
features      = emailFeatures(word_indices)

# Imprima estadísticas
print('\nLength of feature vector: %d' % len(features))
print('Number of non-zero entries: %d' % sum(features > 0))

### 2.3 Entrenamiento de SVM con clasificación de mensajes no deseados (spam)

En la siguiente sección cargamos los datos de entrenamiento que se usaran para el clasificador SVM. El archivo `spamTrain.mat`  (en el directorio
 `Data`  de este ejercicio) contiene 4000 muestras de entrenamiento de mensajes deseados y no deseados, mientras que 
`spamTest.mat` contiene 1000 muestras. Cada mensaje original fue procesado con  las funciones  `processEmail`  y  `emailFeatures` convertidas al vector  $x^{(i)} \in \mathbb{R}^{1899}$.

Luego de cargar los datos, la próxima celda entrena el clasificador SVM entre no deseados ($y=1$) y deseados ($y=0$). Una vez el entrenamiento se haga deberia
ver que el clasificador tiene una exactitud (accuracy) para el entrenamiento cerca al 99.8% y para prueba de 98.5%.

In [ ]:
# Cargue los datos de "Spam Email"
# Debe obtener X, y en su ambiente
data = loadmat(os.path.join('Data', 'spamTrain.mat'))
X, y= data['X'].astype(float), data['y'][:, 0]

print('Training Linear SVM (Spam Classification)')
print('This may take 1 to 2 minutes ...\n')

C = 0.1
model = utils.svmTrain(X, y, C, utils.linearKernel)

In [ ]:
# Calcule la exactitud en el entrenamiento
p = utils.svmPredict(model, X)

print('Training Accuracy: %.2f' % (np.mean(p == y) * 100))

Ejecute la siguiente celda para cargar los datos de prueba y calcular la exactitud (accuracy).


In [ ]:
# Cargue los datos
# debe obtner Xtest, ytest en su ambiente 
data = loadmat(os.path.join('Data', 'spamTest.mat'))
Xtest, ytest = data['Xtest'].astype(float), data['ytest'][:, 0]

print('Evaluating the trained Linear SVM on a test set ...')
p = utils.svmPredict(model, Xtest)

print('Test Accuracy: %.2f' % (np.mean(p == ytest) * 100))

### 2.4 Los predictores sobresalientes

Pare mejor entender como trabaja el clasificador de mensajes no deseados podemos inspeccionar los parámetros para ver que palabras piensa el calsificador que son
mas predictivas. La próxima celda encuentra los parámetros con los mayores valores positivos en el clasificador y muestra las palabras
correspondientes de forma similar a como se muestra en la figura que sigue. 


<div style="border-style: solid; border-width: 1px; margin: 10px 10px 10px 10px; padding: 10px 10px 10px 10px">
our  click  remov guarante visit basenumb dollar pleas price will nbsp most lo ga hour
</div>

En este sentido, si su mensaje contiene palabras como  “guarantee”, “remove”, “dollar”, y “price” (los predictores sobresalientes en la figura), 
es probale que el mensaje sea no deseado (spam). 

Dado que el modelo que estamos entrenando es lineal SVM, podemos inspeccionar los pesos aprendidos por el model para entender mejor como se hizo la clasificación. 
El código a continuación encuentra las palabras con los mayores pesos en el clasificador. Informalmente el clasificador "piensa" que estas palabras son los
mejores indicadores de correos no deseados (spam). 

In [ ]:
# Ordene (sort) los pesos y obtenga el vocabulario
# observe que algunas palabras tienen el mismo peso,
# en este sentido el orden puede ser distinto al texto arriba.

idx = np.argsort(model['w'])
top_idx = idx[-15:][::-1]
vocabList = utils.getVocabList()

print('Top predictors of spam:')
print('%-15s %-15s' % ('word', 'weight'))
print('----' + ' '*12 + '------')
for word, w in zip(np.array(vocabList)[top_idx], model['w'][top_idx]):
    print('%-15s %0.2f' % (word, w))


### 2.5 Opcional: Este ejercicio no es calificado. Trate sus propios "emails".

Ahora que usted a entrenado su clasificador de mensajes, puede tratar con sus propios mensajes. En el código, para comenzar, incluimos ejemplos  (`emailSample1.txt` y `emailSample2.txt`) y dos
ejemplos de mensajes no deseados  (`spamSample1.txt` y `spamSample2.txt`). La próxima celda corre el clasificador sobre el primer grupo y clasifica los mensajes usando lo que
aprendión de SVM. Usted debe tratar con otros ejemplos que suministramos acá y ver si el clasificador acierta. Usted puede tratar con sus propios emails reemplazando los
ejemplos con archivos de texto que contienen sus mensajes. 

*No necesita proveer soluciones por este ejercico que es opcional.*


In [ ]:
filename = os.path.join('Data', 'emailSample1.txt')

with open(filename) as fid:
    file_contents = fid.read()

word_indices = processEmail(file_contents, verbose=False)
x = emailFeatures(word_indices)
p = utils.svmPredict(model, x)

print('\nProcessed %s\nSpam Classification: %s' % (filename, 'spam' if p else 'not spam'))

### 2.6 Opcional: Este ejercicio no es calificado.: Construya su propio conjunto de datos

En este ejercicio suministramos datos de entrenamiento y prueba preprocesados. Estos datos fueron creados usando las mismas funciones  (`processEmail` y `emailFeatures`) 
las cuales usted acaba de completar. Para este ejercicio opcional (no calificado) usted debe construir sus propios datos usando los mensaje orignales de SpamAssassin Public Corpus.

Su tarea es bajar los archivos originales del sitio público. Luego de extaer los mensajes, debe correr  las funciones `processEmail` y `emailFeatures`  para cada mensaje
y extraer un vector de atributos por mensaje. Esto le permite construir datos  `X`, `y`. 
En este punto debe dividir los datos aleatoriamente en datos de entrenamiento, validación y prueba. 

Mientras construye sus propios datos, recomendamos que construya su propio vocabulario (seleccionando las palabras de que ocurren con más frecuencia) y agregar 
cualquier atributo adicional de su preferencia.  Finalmente, tambien sugerimos el uso de herramientas de optimización sofiticadas para SVM tales como:
 [`LIBSVM`](https://www.csie.ntu.edu.tw/~cjlin/libsvm/) or [`scikit-learn`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm).
